----
### **Deep Learning methodologies**

In [2]:
import os
import numpy as np
import pandas as pd
import pickle
import random
import warnings
from copy import deepcopy
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Flatten,Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
import csv
from tensorflow.keras.models import Model

#import keras
from tensorflow.keras import backend as k
from tensorflow.keras.layers import Input,Concatenate,Dropout,Dense,BatchNormalization,Conv1D
from tensorflow.keras.layers import Input
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

#from tensorflow.keras.merge import concatenate
#import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import scipy
from tensorflow.keras.initializers import he_normal,glorot_normal
from tensorflow.keras.regularizers import l1,l2
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau

from time import time
from tensorflow.keras.utils import plot_model
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras.callbacks import Callback 
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Input,Activation,BatchNormalization,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

import random as rn
import string
from sklearn.metrics import f1_score
from tensorflow import keras
import tensorflow as tf
import numpy as np
import datetime
import os
import math
import pandas as pd
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.initializers import glorot_uniform,glorot_normal
from tensorflow.keras.layers import MaxPooling1D


2023-05-19 07:53:27.209564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False)

Available CPUs: 8
INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
## Loading cleaned data
df_proc=pd.read_csv('/Users/kshitijmittal/Documents/UChicago Acad/03 Quarter 3/01 ML/Essay_Grading_DL/data/preprocessed-essay.csv')
df_proc.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,cleaned_tokenize_text,corrected_text,...,sent_count,sent_len,mistakes,num_mistakes,word_count,noun_count,verb_count,adjective_count,adverb_count,Score_Category
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,think students would benefit learning home won...,think students would benefit learning home won...,...,18,14.500000,NaN,0,261,61,25,25,5,medium
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,problem change let best matter happening chang...,problem change let best matter happening chang...,...,14,38.071429,NaN,0,533,100,56,37,9,medium
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,dear principal change school policy grade aver...,dear principal change school policy grade aver...,...,19,16.842105,NaN,0,320,58,20,33,5,medium
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,best time life become agree greatest accomplis...,best time life become agree greatest accomplis...,...,36,20.222222,NaN,0,728,106,73,62,22,high
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,small act kindness impact people change people...,small act kindness impact people change people...,...,3,78.000000,kindess homeles triying greate kindess wount t...,11,234,58,21,18,5,medium


In [70]:
df_proc['lemmatized_text']

0       think student would benefit learn home wont ch...
1       problem change let best matter happen change m...
2       dear principal change school policy grade aver...
3       best time life become agree great accomplishme...
4       small act kindness impact people change people...
                              ...                        
3906    believe use cellphone class education goodbye ...
3907    work alone student argue decision project work...
3908    problem chance best think quote cant best ever...
3909    many people disagree albert schweitzer quote e...
3910    think failure main thing people consist goal s...
Name: lemmatized_text, Length: 3911, dtype: object

In [33]:
## Checking average word count in the essays
df_proc['word_count'].describe()

count    3911.000000
mean      430.492969
std       191.868449
min        14.000000
25%       294.000000
50%       402.000000
75%       526.500000
max      1260.000000
Name: word_count, dtype: float64

In [115]:
y=df_proc[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']]
X=df_proc['lemmatized_text']

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.10)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(3519,) (3519, 6)
(392,) (392, 6)


In [128]:
X_train

585     certain school district allow program high sch...
2713    former british prime minister winston churchil...
1039    topic brings lot question concern like charact...
860     teenager really complicate day try extra hour ...
1540    good prescient procession prescient many educa...
                              ...                        
1453    ever cross mind people today struggle make dec...
3741    think enjoyable educational activity gym gym f...
2667    go talk prompt tomas jefferson write idle alwa...
3399    would like consider hire job responsible quick...
975     problem lead success try best problem chance b...
Name: lemmatized_text, Length: 3519, dtype: object

In [129]:
X_test

453     imagine school plan add time school disagree s...
110     character choose make word british naturalist ...
1816    think section everyone smart want graduation t...
2962    character trait form influence beyond control ...
3652    positive attitude key success well opinion str...
                              ...                        
2852    conserve energy resource work foundry week job...
1324    agree british prime minister winston churchill...
2595    genericname promptly prompt explain people see...
29      agree everyone make decision one life permit p...
265     good idea student finish high school three yea...
Name: lemmatized_text, Length: 392, dtype: object

In [117]:
## Adding code for padding
def pad_text(text,tokenizer,max_len):
    return pad_sequences(tokenizer.texts_to_sequences(text),maxlen=max_len,padding='post')

In [118]:
from tensorflow.keras.preprocessing.text import Tokenizer

def text_padding(train,test,max_len):
    token=Tokenizer()
    token.fit_on_texts(train)
    padded_train_text=pad_text(train,token,max_len)
    padded_test_text=pad_text(test,token,max_len)
    # padded_test_final_text=pad_text(test_final,token,max_len)
    return padded_train_text,padded_test_text,token

In [119]:
max_len=500
train_com_pad,test_com_pad,token_com= text_padding(X_train,X_test, max_len)

In [120]:
train_com_pad

array([[ 461,    3,  971, ...,    0,    0,    0],
       [1255,  797,  866, ...,    0,    0,    0],
       [ 421,  972,   35, ...,    0,    0,    0],
       ...,
       [  23,   75,  650, ...,    0,    0,    0],
       [  14,   11,  528, ...,    0,    0,    0],
       [  73,  278,   72, ...,    0,    0,    0]], dtype=int32)

In [121]:
test_com_pad

array([[ 290,    3,  265, ...,    0,    0,    0],
       [ 156,  107,    6, ...,    0,    0,    0],
       [  15, 2096,  100, ...,    0,    0,    0],
       ...,
       [ 112,  650,  356, ...,    0,    0,    0],
       [  96,  100,    6, ...,    0,    0,    0],
       [   9,   48,    1, ...,    0,    0,    0]], dtype=int32)

In [122]:
token_com
num_words = len(token_com.word_index)+1
num_words

9809

In [124]:
print(token_com.word_index)

{'student': 1, 'people': 2, 'school': 3, 'work': 4, 'get': 5, 'make': 6, 'time': 7, 'want': 8, 'good': 9, 'help': 10, 'like': 11, 'life': 12, 'thing': 13, 'would': 14, 'think': 15, 'class': 16, 'one': 17, 'well': 18, 'take': 19, 'need': 20, 'know': 21, 'learn': 22, 'go': 23, 'also': 24, 'something': 25, 'person': 26, 'example': 27, 'say': 28, 'way': 29, 'day': 30, 'try': 31, 'always': 32, 'give': 33, 'many': 34, 'lot': 35, 'use': 36, 'reason': 37, 'teacher': 38, 'friend': 39, 'change': 40, 'first': 41, 'could': 42, 'home': 43, 'job': 44, 'bad': 45, 'do': 46, 'year': 47, 'idea': 48, 'see': 49, 'high': 50, 'really': 51, 'others': 52, 'hard': 53, 'new': 54, 'attitude': 55, 'feel': 56, 'positive': 57, 'important': 58, 'career': 59, 'even': 60, 'technology': 61, 'someone': 62, 'come': 63, 'never': 64, 'family': 65, 'best': 66, 'different': 67, 'play': 68, 'start': 69, 'world': 70, 'future': 71, 'success': 72, 'problem': 73, 'online': 74, 'talk': 75, 'might': 76, 'much': 77, 'group': 78, 'st

-----------
### **Part1: Baseline Modeling - RNNs**

Only picking up 1 metric - cohesion

In [130]:
from keras import layers
base_rnn=keras.Sequential(
    [layers.Embedding(input_dim=num_words, output_dim=100, input_length=max_len), ## Only the first layer needs an input dimension
     # We have 9841 total words (num_words). We are converting them into dense vectors of size output_dim. Input length is constant, padded at 500
     layers.SimpleRNN(units=32, activation='relu'),
     # Output of SimpleRNN is batch size, number of units; so in our case 128 rows, 32 columns
     # What does number of units mean? 
     # How many time steps will we have, how many features?
     layers.Dense(units=1, activation='linear')
    ]
)

base_rnn.compile(
    loss='mse',
    optimizer=keras.optimizers.Adam(),
    metrics='mse'
)

base_rnn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 100)          980900    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 32)                4256      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 985,189
Trainable params: 985,189
Non-trainable params: 0
_________________________________________________________________


In [131]:
X_train_tens=tf.convert_to_tensor(train_com_pad, dtype=tf.int32)
X_test_tens=tf.convert_to_tensor(test_com_pad, dtype=tf.int32)
X_train_tens.shape

TensorShape([3519, 500])

In [132]:
y_train_tens=tf.convert_to_tensor(y_train['cohesion'], dtype=tf.int32)
y_test_tens=tf.convert_to_tensor(y_test['cohesion'], dtype=tf.int32)
y_train_tens.shape

TensorShape([3519])

In [133]:
%%time

# Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.
history = base_rnn.fit(x=X_train_tens, y=y_train_tens, validation_split=0.25, epochs=20, batch_size=128)

Epoch 1/20
21/21 [==============================] - 3s 119ms/step - loss: 6.4270 - mse: 6.4270 - val_loss: 2.1055 - val_mse: 2.1055
Epoch 2/20
21/21 [==============================] - 2s 105ms/step - loss: 0.8444 - mse: 0.8444 - val_loss: 0.6012 - val_mse: 0.6012
Epoch 3/20
21/21 [==============================] - 2s 106ms/step - loss: 0.5357 - mse: 0.5357 - val_loss: 0.5604 - val_mse: 0.5604
Epoch 4/20
21/21 [==============================] - 2s 108ms/step - loss: 0.5060 - mse: 0.5060 - val_loss: 0.5413 - val_mse: 0.5413
Epoch 5/20
21/21 [==============================] - 2s 107ms/step - loss: 0.5063 - mse: 0.5063 - val_loss: 0.5413 - val_mse: 0.5413
Epoch 6/20
21/21 [==============================] - 2s 103ms/step - loss: 0.4982 - mse: 0.4982 - val_loss: 0.5370 - val_mse: 0.5370
Epoch 7/20
21/21 [==============================] - 2s 101ms/step - loss: 0.4982 - mse: 0.4982 - val_loss: 0.5342 - val_mse: 0.5342
Epoch 8/20
21/21 [==============================] - 2s 102ms/step - loss: 0.

In [134]:
base_rnn.evaluate(
    x = X_test_tens, y = y_test_tens, batch_size=128, use_multiprocessing = True
    )

4/4 [==============================] - 0s 19ms/step - loss: 0.5311 - mse: 0.5311


[0.5311155915260315, 0.5311155915260315]

-----
### **Part1: Baseline Modeling - LSTMS**


In [136]:
model2=keras.Sequential([
    layers.Embedding(input_dim=num_words, output_dim=500, input_length=max_len),
    layers.LSTM(units=64, recurrent_dropout=0.5),
    layers.Dropout(rate=0.5),
    layers.Dense(units=1, activation='linear')
])

model2.compile(
    loss='mse',
    optimizer=keras.optimizers.Adam(),
    metrics='mse'
)

model2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 500, 500)          4904500   
                                                                 
 lstm (LSTM)                 (None, 64)                144640    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,049,205
Trainable params: 5,049,205
Non-trainable params: 0
_________________________________________________________________


In [137]:
# Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.
model2.fit(x=X_train_tens, y=y_train_tens, validation_split=0.25, epochs=20, batch_size=128, use_multiprocessing=True)

Epoch 1/20
21/21 [==============================] - 17s 747ms/step - loss: 2.7130 - mse: 2.7130 - val_loss: 0.5499 - val_mse: 0.5499
Epoch 2/20
21/21 [==============================] - 14s 678ms/step - loss: 0.8083 - mse: 0.8083 - val_loss: 0.5633 - val_mse: 0.5633
Epoch 3/20
21/21 [==============================] - 16s 791ms/step - loss: 0.7324 - mse: 0.7324 - val_loss: 0.5268 - val_mse: 0.5268
Epoch 4/20
21/21 [==============================] - 16s 780ms/step - loss: 0.7164 - mse: 0.7164 - val_loss: 0.5253 - val_mse: 0.5253
Epoch 5/20
21/21 [==============================] - 17s 783ms/step - loss: 0.6754 - mse: 0.6754 - val_loss: 0.5247 - val_mse: 0.5247
Epoch 6/20
21/21 [==============================] - 14s 689ms/step - loss: 0.6621 - mse: 0.6621 - val_loss: 0.5242 - val_mse: 0.5242
Epoch 7/20
21/21 [==============================] - 14s 682ms/step - loss: 0.6721 - mse: 0.6721 - val_loss: 0.5321 - val_mse: 0.5321
Epoch 8/20
21/21 [==============================] - 17s 794ms/step - 

In [138]:
model2.evaluate(x = X_test_tens, y = y_test_tens, batch_size=128, use_multiprocessing = True)

4/4 [==============================] - 1s 132ms/step - loss: 0.5324 - mse: 0.5324


[0.5324367880821228, 0.5324367880821228]